# Halloween Candy Survey - Data Cleaning and Preparation

## Overview

The purpose of this project was to clean and wrangle data from a survey of Halloween candy to prepare it for a machine learning project. This has been modified from a data cleaning assignment completed for my Master's program.


## Data Set

The data set used is the 2017 Halloween Candy Hierarchy data set as discussed in this [boingboing](https://boingboing.net/2017/10/30/the-2017-halloween-candy-hiera.html) article.

The following are the rating instructions from the survey:  

> Basically, consider that feeling you get when you receive this item in your Halloween haul. Does it make you really happy (JOY)? Or is it something that you automatically place in the junk pile (DESPAIR)? MEH for indifference, and you can leave blank if you have no idea what the item is.

This dataset provided an opportunity to work with very messy data, as respondents were allowed to enter unconstrained text for a number of the fields. 


## Our End Goal

The end goal for this project is to clean the data so that it could be used to create a machine learning model in the future. We want to see if we are able to predict a person's gender based purely on their candy preferences. Note: a model was not created in this project. The data was simply cleaned and prepared for use.

## Initial Import & Exploration

In [ ]:
# initial imports
import pandas as pd
import numpy as np

Data was first imported and a DataFrame was created called `candy`. `encoding='iso-8859-1'` was used during the import because of special characters in the some respondent answers that were not recognized by Pandas. 

In [ ]:
# read_csv with iso-8859-1 encoding
candy_full = pd.read_csv('candy.csv', encoding='iso-8859-1')

candy = candy_full.copy()

I first looked at the data column types using `head()`.

In [ ]:
# first five rows
candy.head()

Then, looked at information about the DataFrame.

In [ ]:
# check info about the DataFrame
candy.info()

Looking at column names:

In [ ]:
# print a list of column names
for col in candy.columns:
    print(col)

## Data Cleaning

I began cleaning by removing the character `Õ`, which is meant to be an apostrophe `'`.

In [ ]:
candy.rename({'Q6 | Chick-o-Sticks (we donÕt know what that is)' : "Q6 | Chick-o-Sticks (we don't know what that is)", 
              'Q6 | HersheyÕs Milk Chocolate' : "Q6 | Hershey's Milk Chocolate", 
              'Q6 | Peanut M&MÕs' : "Q6 | Peanut M&M's", 
              'Q6 | ReeseÕs Peanut Butter Cups' : "Q6 | Reese's Peanut Butter Cups"}, 
            axis = 1, inplace = True)

for col in candy.columns:
    print(col)

I then determined how many duplicated rows there were in the file, assuming that a duplicate is any row with the same `Internal ID` number as another.

In [1]:
dup = candy['Internal ID'].duplicated().sum()
dup

NameError: name 'candy' is not defined

Duplicates were then dropped from the DataFrame.

In [ ]:
candy.drop_duplicates(['Internal ID'], inplace = True)
print(candy.shape)

The following irrelevant columns were then removed from the DataFrame:
`Internal ID`, `Q5: STATE, PROVINCE, COUNTY, ETC`, `Q7: JOY OTHER`, `Q8: DESPAIR OTHER`, `Q9: OTHER COMMENTS`, `Unnamed: 113`, `Click Coordinates (x, y)`.

In [ ]:
drop_cols = ['Internal ID', 'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q7: JOY OTHER', 'Q8: DESPAIR OTHER', 
             'Q9: OTHER COMMENTS', 'Unnamed: 113', 'Click Coordinates (x, y)']

candy = candy.drop(drop_cols, axis = 1)
print(candy.shape)

Next, the `Q2: GENDER` column was explored, as this will be the response variable in the machine learning model. Value counts and missing values are analyzed.

In [ ]:
candy['Q2: GENDER'].value_counts()

In [ ]:
missing_gender = candy['Q2: GENDER'].isnull().sum()
missing_gender

All rows with a missing value in the `Q2: GENDER` column were removed.

In [ ]:
candy = candy.dropna(subset = ['Q2: GENDER'])
candy['Q2: GENDER'].isnull().sum()

We want to predict between Male or Female in this model. Because of this, only the rows that contain either Male or Female in the `Q2: GENDER` column were selected.

In [ ]:
candy = candy.drop(candy[candy['Q2: GENDER'] == "I'd rather not say"].index)
candy = candy.drop(candy[candy['Q2: GENDER'] == "Other"].index)
candy['Q2: GENDER'].value_counts()

Missing values in the `Q1: GOING OUT?` were then evaluated, as we want individuals who did go out on Halloween.

In [ ]:
missing_out = candy['Q1: GOING OUT?'].isnull().sum()
missing_out

Any missing values were then filled in with a `No` response.

In [ ]:
candy['Q1: GOING OUT?'] = candy['Q1: GOING OUT?'].fillna('No')
candy['Q1: GOING OUT?'].value_counts()

To get ready for the next step, I sliced a subset of columns for cleaning: `Q6 | 100 Grand Bar` to `Q11: DAY`.

In [ ]:
candy_slice = candy.loc[:, 'Q6 | 100 Grand Bar':'Q11: DAY']
for col in candy_slice.columns:
    print(col)

I then filled any missing values in the `candy` DataFrame for those columns with the string `NO_ANSWER`.

In [ ]:
candy.loc[:, 'Q6 | 100 Grand Bar':'Q11: DAY'] = candy_slice.fillna('NO_ANSWER')
candy.info(max_cols=120)

For all four `Q12: Media` columns, I filled the missing values with `0.0`.

In [ ]:
candy.loc[:, 'Q12: MEDIA [Daily Dish]':'Q12: MEDIA [Yahoo]'] = candy.loc[:, 'Q12: MEDIA [Daily Dish]':'Q12: MEDIA [Yahoo]'].fillna(0.0)
candy.loc[:, 'Q12: MEDIA [Daily Dish]':'Q12: MEDIA [Yahoo]'].isnull().sum()

The next major column to address, which is filled with very messy data, provides the countries of respondents.

In [ ]:
# check unique values
candy['Q4: COUNTRY'].unique()

I wanted to clean up this data to only include four areas: USA, Canada, Europe, and Other.

In [ ]:
# adjust any null values to 'Other'
candy.loc[:, 'Q4: COUNTRY'] = candy.loc[:, 'Q4: COUNTRY'].fillna('Other')

# combine Australia values into 'Other'
candy['Q4: COUNTRY'].replace({
    'australia' : 'Other',
    'Australia' : 'Other'
}, inplace = True)

In [ ]:
# combine all 'USA' entries
candy['Q4: COUNTRY'].replace({
    'USA ' : 'USA',
    'us' : 'USA',
    'usa' : 'USA', 
    'Us' : 'USA', 
    'US' : 'USA', 
    'Murica' : 'USA', 
    'United States' : 'USA', 
    'united states' : 'USA', 
    'Usa' : 'USA', 
    'United States ' : 'USA', 
    'United staes' : 'USA', 
    'United States of America' : 'USA', 
    'United states' : 'USA', 
    'u.s.a.' : 'USA', 
    'United States of America ' : 'USA', 
    'america' : 'USA', 
    'U.S.A.' : 'USA', 
    'unhinged states' : 'USA', 
    'united states of america' : 'USA', 
    'US of A' : 'USA', 
    'The United States' : 'USA', 
    'North Carolina ' : 'USA', 
    'Unied States' : 'USA', 
    'U S' : 'USA', 
    'u.s.' : 'USA', 
    'The United States of America' : 'USA', 
    'unite states' : 'USA',
    'U.S.' : 'USA', 
    'USA? Hard to tell anymore..' : 'USA', 
    "'merica" : 'USA', 
    'United State' : 'USA', 
    'United Sates' : 'USA', 
    'California' : 'USA', 
    'Unites States' : 'USA', 
    'USa' : 'USA', 
    'I pretend to be from Canada, but I am really from the United States.' : 'USA', 
    'Usa ' : 'USA', 
    'United Stated' : 'USA', 
    'New Jersey' : 'USA', 
    'United ststes' : 'USA', 
    'America' : 'USA', 
    'United Statss' : 'USA', 
    'murrika' : 'USA', 
    'USA! USA! USA!' : 'USA', 
    'USAA' : 'USA', 
    'united States ' : 'USA', 
    'N. America' : 'USA', 
    'USSA' : 'USA', 
    'U.S. ' : 'USA', 
    'u s a' : 'USA', 
    'United Statea' : 'USA', 
    'united ststes' : 'USA', 
    'USA USA USA!!!!' : 'USA'
}, inplace = True)

In [ ]:
# combine all Canada entries as 'CA'

candy['Q4: COUNTRY'].replace({
    'canada' : 'CA', 
    'Canada' : 'CA', 
    'canada ' : 'CA', 
    'Canada ' : 'CA', 
    'Can' : 'CA', 
    'Canae' : 'CA', 
    'Canada`' : 'CA', 
    'CANADA' : 'CA'
}, inplace = True)

In [ ]:
# combine all Europe entries as 'EU'

candy['Q4: COUNTRY'].replace({
   'uk' : 'EU', 
    'United Kingdom' : 'EU', 
    'England' : 'EU', 
    'UK' : 'EU', 
    'france' : 'EU', 
    'finland' : 'EU', 
    'Netherlands' : 'EU', 
    'germany' : 'EU', 
    'Europe' : 'EU', 
    'U.K. ' : 'EU', 
    'Greece' : 'EU', 
    'France' : 'EU', 
    'Ireland' : 'EU', 
    'Uk' : 'EU', 
    'Germany' : 'EU', 
    'Scotland' : 'EU', 
    'UK ' : 'EU', 
    'Denmark' : 'EU', 
    'France ' : 'EU', 
    'Switzerland' : 'EU', 
    'Scotland ' : 'EU', 
    'The Netherlands' : 'EU', 
    'Ireland ' : 'EU', 
    'spain' : 'EU', 
    'Sweden' : 'EU', 
    'United kingdom' : 'EU'
}, inplace = True)

In [ ]:
# change any remaining entries to 'Other'

candy.loc[~candy['Q4: COUNTRY'].isin(['USA', 'CA', 'EU']), 'Q4: COUNTRY'] = 'Other'

# code check
candy['Q4: COUNTRY'].unique()

Next, the `Age` column was addressed. 

In [ ]:
# check unique age values
candy['Q3: AGE'].unique()

These values were placed into the following categorical bins: `unknown`, `17 and under`, `18-25`, `26-35`, `36-45`, `46-55`, and `56+`. Any text values were binned with `unknown` for ease in this project. Categories were then reindexed in order.

In [ ]:
# create True/False index
age_index = candy['Q3: AGE'].str.isnumeric()

# for the index, fill missing values with False
age_index = age_index.fillna(False)

# select Age column for only those False values from index and code as missing
candy.loc[~age_index, 'Q3: AGE'] = np.nan

candy['Q3: AGE'] = candy['Q3: AGE'].astype('float')

candy['Q3: AGE'] = pd.cut(x = candy['Q3: AGE'], bins = [-1, 17, 25, 35, 45, 55, 100], 
                         labels = ['17 and under', '18-25', '26-35', '36-45', '46-55', '56+'])

candy['Q3: AGE'] = candy['Q3: AGE'].cat.add_categories('unknown')
candy['Q3: AGE'] = candy['Q3: AGE'].cat.as_unordered()
candy['Q3: AGE'] = candy['Q3: AGE'].cat.reorder_categories(
    ['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'])
candy.loc[:, 'Q3: AGE'] = candy.loc[:, 'Q3: AGE'].fillna('unknown')

# double check categories
candy['Q3: AGE'].cat.categories

Final checks were performed to ensure there were no missing values, and the index was reset for easier grading in the original assignment.

In [ ]:
# missing value check
candy.isnull().sum()

# index reset
candy = candy.reset_index(drop = True)
#candy.info(max_cols=120)

## Feature Engineering

In this section, a new column was created called "net_feelies" (calculated by the authors as the total joy count minus the total despair count).

In [ ]:
# data sliced for easier calculations
candy_reduced = candy.loc[:, 'Q6 | 100 Grand Bar':'Q6 | York Peppermint Patties']

Next, I created two Series, one with JOY counts and one with DESPAIR counts to add to the `candy_reduced` data. `joy_count` lists total counts for JOY for each column, while `despair_count` that lists the total counts for DESPAIR for each column.

In [ ]:
joy_count = pd.Series(candy_reduced[candy_reduced == 'JOY'].count())
despair_count = pd.Series(candy_reduced[candy_reduced == 'DESPAIR'].count())

In [ ]:
# transpose of the candy_reduced DataFrame
candy_reduced_transpose = candy_reduced.T

I added a new column called "joy_count" using the `joy_count` Series above and a new column called 'despair_count" using the `despair_count` Series above to the `candy_reduced_transpose` DataFrame. I then added a new column to the `candy_reduced_transpose` DataFrame called "net_feelies" that takes the `joy_count` column and subtracts the `despair_count` column.

In [ ]:
candy_reduced_transpose['joy_count'] = joy_count
candy_reduced_transpose['despair_count'] = despair_count
candy_reduced_transpose['net_feelies'] = candy_reduced_transpose['joy_count'] - candy_reduced_transpose['despair_count']

This data was then sorted in descending order by `net feelies`.

In [ ]:
candy_net_sorted = candy_reduced_transpose.loc[:, 'joy_count':'net_feelies'].sort_values(
    by = 'net_feelies', ascending = False)

## Encoding

The next step is to get the `candy` DataFrame ready to run a machine learning algorthim to determine if we could predict a person's gender based on what candy they prefer. I turned all of the values into numeric values, using Pandas to perform these exercises.

In [ ]:
# copy of the DataFrame

candy_encode = candy.copy()
#candy_encode

First, I replaced any `Female` values with `0` and any `Male` values with `1`.

In [ ]:
gender_map = {'Female' : 0, 'Male' : 1}

candy_encode['Q2: GENDER'] = candy_encode['Q2: GENDER'].map(gender_map)

#candy_encode.head()

Then, I separateed the column that we want to predict (response) and the columns that will be used to make the predictions (features). 

In [ ]:
candy_response = pd.Series(candy_encode['Q2: GENDER'])

encode_drop = ['Q2: GENDER', 'Q1: GOING OUT?', 'Q3: AGE', 'Q4: COUNTRY', 'Q10: DRESS', 'Q11: DAY', 
               'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]']
candy_features = candy_encode.drop(encode_drop, axis = 1)

candy_features = candy_features.reset_index(drop=True)

Finally, I used Pandas' `get_dummies()` to encode the `candy_features` data.

In [ ]:
candy_features_encoded = pd.get_dummies(candy_features, drop_first = True)
candy_features_encoded = candy_features_encoded.reset_index(drop=True)

At this point, the data is now cleaned, encoded, and ready to be used to create a machine learning model.